In [103]:
import numpy as np
import pandas as pd
import itertools
import time
import os
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error


In [104]:
# store = 5136
stores = [1096, 2177, 2189, 3328, 3349, 3889, 4388, 4816, 5136]
product = 8011605
filtered_items = [8011001, 8011003, 8010903, 8011740, 8011002, 8017442, 8011605]

In [105]:
for store in stores:
    data_2017 = f"Magaza CSV Files 2017/magaza_{store}_2017.csv"
    data_2018 = f"Magaza CSV Files 2018/magaza_{store}_2018.csv"
    data_2019 = f"Magaza CSV Files 2019/magaza_{store}_2019.csv"

    df_2017 = pd.read_csv(data_2017)
    df_2018 = pd.read_csv(data_2018)
    df_2019 = pd.read_csv(data_2019)

    df = pd.concat([df_2017, df_2018], axis=0)

    df.sort_values("tarih", inplace=True)
    df.set_index("tarih", inplace=True)
    time_series = df["toplam_satis_adet"] #time_series df of sales

    def optimize_arima(time_series, p_values, d_values, q_values):

        best_score = float("inf")
        best_cfg = None
        best_model = None

        train_size = int(len(time_series) * 0.8)
        train, test = time_series[:train_size], time_series[train_size:]

        for p, d, q in itertools.product(p_values, d_values, q_values):
            try:
                model = ARIMA(train, order=(p, d, q))
                model_fit = model.fit()
                forecast = model_fit.forecast(steps=len(test))
                rmse = np.sqrt(mean_squared_error(test, forecast))
                if rmse < best_score:
                    best_score = rmse
                    best_cfg = (p, d, q)
                    best_model = model_fit
            except Exception as e:
                print(f"ARIMA({p},{d},{q}) model could not be formed: {e}")
                continue

        forecast = best_model.forecast(steps=len(test))
        rmse_final = np.sqrt(mean_squared_error(test, forecast))
        mape_final = mean_absolute_percentage_error(test, forecast) * 100
        return best_cfg, best_model, rmse_final, mape_final

    p_values = [0, 1, 2]
    d_values = [0, 1]
    q_values = [0, 1, 2]

    start_time = time.time()
    best_cfg, best_model, rmse_final, mape_final = optimize_arima(time_series, p_values, d_values, q_values)
    model_runtime = time.time() - start_time

    print(f"Best parameters: {best_cfg}")
    print(f"Test RMSE: {rmse_final}")
    print(f"Test MAPE: {mape_final}")
    print(f"Model runtime: {np.round(model_runtime, 3)} seconds")

    metrics_file = "metrics_log.csv"
    if os.path.exists(metrics_file):
        metrics_df = pd.read_csv(metrics_file)
    else:
        metrics_df = pd.DataFrame(columns=["Product", "Store", "RMSE", "MAPE", "Model Runtime", "Model"])

    new_row = {
        "Product": product,
        "Store": store,
        "RMSE": rmse_final,
        "MAPE": mape_final,
        "Model Runtime": model_runtime,
        "Model": "ARIMA",
        "Epochs": 1
    }

    new_row_df = pd.DataFrame([new_row])
    metrics_df = pd.concat([metrics_df, new_row_df], ignore_index=True)

    metrics_df.to_csv(metrics_file, index=False)
    print("Metrics successfully saved!")

    # # # # # ## # ## # ## # ## # ## # ## # ## # ## # ## # ## # ## # ## # ## # ## # ## # ## # ## # ## # #

    p, d, q = best_cfg
    print(f"p: {p}, d: {d}, q: {q}")

    # # # # # ## # ## # ## # ## # ## # ## # ## # ## # ## # ## # ## # ## # ## # ## # ## # ## # ## # ## # #

    def optimize_sarima(time_series, p, d, q, P_values, D_values, Q_values, s_values):

        best_score = float("inf")
        best_cfg = None
        best_model = None

        train_size = int(len(time_series) * 0.8)
        train, test = time_series[:train_size], time_series[train_size:]

        for P, D, Q, s in itertools.product(P_values, D_values, Q_values, s_values):
            try:
                seasonal_order = (P, D, Q, s)
                model = SARIMAX(train, order=(p, d, q), seasonal_order=seasonal_order,
                                enforce_stationarity=False, enforce_invertibility=False)
                model_fit = model.fit(disp=False)
                forecast = model_fit.forecast(steps=len(test))
                rmse = np.sqrt(mean_squared_error(test, forecast))
                if rmse < best_score:
                    best_score = rmse
                    best_cfg = (p, d, q, seasonal_order)
                    best_model = model_fit
            except Exception as e:
                print(f"SARIMA({p},{d},{q})x({P},{D},{Q},{s}) model could not be formed: {e}")
                continue

        forecast = best_model.forecast(steps=len(test))
        rmse_final = np.sqrt(mean_squared_error(test, forecast))
        mape_final = mean_absolute_percentage_error(test, forecast) * 100
        return best_cfg, best_model, rmse_final, mape_final

    P_values = [0, 1]
    D_values = [0, 1]
    Q_values = [0, 1]
    s_values = [7, 14, 30, 60, 90]

    start_time = time.time()
    best_cfg, best_model, rmse_final, mape_final = optimize_sarima(time_series, p, d, q,
                                                                    P_values, D_values, Q_values, s_values)
    model_runtime = time.time() - start_time

    print(f"Best parameters: {best_cfg}")
    print(f"Test RMSE: {rmse_final}")
    print(f"Test MAPE: {mape_final}")
    print(f"Model runtime: {np.round(model_runtime, 3)} seconds")

    metrics_file = "metrics_log.csv"

    if os.path.exists(metrics_file):
        metrics_df = pd.read_csv(metrics_file)
    else:
        metrics_df = pd.DataFrame(columns=["Product", "Store", "RMSE", "MAPE", "Model Runtime", "Model"])

    new_row = {
        "Product": product,
        "Store": store,
        "RMSE": rmse_final,
        "MAPE": mape_final,
        "Model Runtime": model_runtime,
        "Model": "SARIMA",
        "Epochs": 1
    }

    new_row_df = pd.DataFrame([new_row])
    metrics_df = pd.concat([metrics_df, new_row_df], ignore_index=True)

    metrics_df.to_csv(metrics_file, index=False)
    print("Metrics successfully saved!")

/Users/mertgoker/Documents/GitHub/Senior-Design-Project/.venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/mertgoker/Documents/GitHub/Senior-Design-Project/.venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/mertgoker/Documents/GitHub/Senior-Design-Project/.venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/mertgoker/Documents/GitHub/Senior-Design-Project/.venv/lib/python3.11/site-packages/statsm

Best parameters: (0, 0, 0)
Test RMSE: 3.08645989062659
Test MAPE: 68.9206282967992
Model runtime: 0.366 seconds
Metrics successfully saved!
p: 0, d: 0, q: 0


/Users/mertgoker/Documents/GitHub/Senior-Design-Project/.venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/mertgoker/Documents/GitHub/Senior-Design-Project/.venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/Users/mertgoker/Documents/GitHub/Senior-Design-Project/.venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/mertgoker/Documents/GitHub/Senior-Design-Project/.venv/lib/python3.11/site-packages/statsmo

Best parameters: (0, 0, 0, (0, 1, 1, 60))
Test RMSE: 3.099633225310606
Test MAPE: 69.50888631848112
Model runtime: 23.515 seconds
Metrics successfully saved!


/Users/mertgoker/Documents/GitHub/Senior-Design-Project/.venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/mertgoker/Documents/GitHub/Senior-Design-Project/.venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/mertgoker/Documents/GitHub/Senior-Design-Project/.venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/mertgoker/Documents/GitHub/Senior-Design-Project/.venv/lib/python3.11/site-packages/statsm

...

/Users/mertgoker/Documents/GitHub/Senior-Design-Project/.venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/mertgoker/Documents/GitHub/Senior-Design-Project/.venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/mertgoker/Documents/GitHub/Senior-Design-Project/.venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/mertgoker/Documents/GitHub/Senior-Design-Project/.venv/lib/python3.11/site-packages/statsmodels/tsa/base/t

Best parameters: (2, 1, 1)
Test RMSE: 1.3462912017398831
Test MAPE: 58.958333350091074
Model runtime: 0.382 seconds
Metrics successfully saved!
p: 2, d: 1, q: 1


/Users/mertgoker/Documents/GitHub/Senior-Design-Project/.venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/mertgoker/Documents/GitHub/Senior-Design-Project/.venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/Users/mertgoker/Documents/GitHub/Senior-Design-Project/.venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/mertgoker/Documents/GitHub/Senior-Design-Project/.venv/lib/python3.11/site-packages/statsmo

Best parameters: (2, 1, 1, (0, 1, 1, 7))
Test RMSE: 1.3122825019956488
Test MAPE: 66.66674210554928
Model runtime: 45.974 seconds
Metrics successfully saved!


/Users/mertgoker/Documents/GitHub/Senior-Design-Project/.venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/mertgoker/Documents/GitHub/Senior-Design-Project/.venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/Users/mertgoker/Documents/GitHub/Senior-Design-Project/.venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/mertgoker/Documents/GitHub/Senior-Design-Project/.venv/lib/python3.11/site-packages/statsmodels/tsa/base/ts